In [2]:
import pandas as pd
#import keepsake
import numpy as np
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.style.use('ggplot')

%matplotlib inline

In [3]:
# only need to be run once to initialize

# ! echo 'repository: "file://.keepsake"' > keepsake.yaml 

In [3]:
# Generate signal combinations
from itertools import combinations

comb_list = []

for features in [1,2,3]:
    for com in combinations(['gyro_x','gyro_y','gyro_z'], features):
        comb_list.append(f'acc_x|acc_y|acc_z|{"|".join(list(com))}|label')
comb_list

['acc_x|acc_y|acc_z|gyro_x|label',
 'acc_x|acc_y|acc_z|gyro_y|label',
 'acc_x|acc_y|acc_z|gyro_z|label',
 'acc_x|acc_y|acc_z|gyro_x|gyro_y|label',
 'acc_x|acc_y|acc_z|gyro_x|gyro_z|label',
 'acc_x|acc_y|acc_z|gyro_y|gyro_z|label',
 'acc_x|acc_y|acc_z|gyro_x|gyro_y|gyro_z|label']

In [4]:
# df_10hz = pd.read_csv('data/transformed/20210529_v2_data_all_10hz.csv') # not neede for experiments
df_20hz = pd.read_csv('data/transformed/20210529_v2_data_all_20hz.csv')
df_25hz = pd.read_csv('data/transformed/20210529_v2_data_all_25hz.csv')
df_50hz = pd.read_csv('data/transformed/20210529_v2_data_all_50hz.csv')
# df_100hz = pd.read_csv('data/transformed/20210529_v2_data_all_100hz.csv') # not neede for experiments

In [5]:
def get_df_base(df):
    '''Gets baseline dataset.'''
    df = df[(df['shift'] == 0)]
    return df.dropna(axis=0)

In [6]:
df_20hz = get_df_base(df_20hz)
df_25hz = get_df_base(df_25hz)
df_50hz = get_df_base(df_50hz)

In [8]:
def save_model_optimized(classifier, stage, dataset, model_type, exp_id):
    '''
    Saves model to defined folder.
    
    Args:
        stage: baseline/optimized
        dataset: base/centered/end/etc
        model_types: decision_tree, random_forest, ...
        hz: frequency
    Returns:
        Saved file path.
    '''

    import os
    import m2cgen as m2c
    
    BASE_PATH = f'models/{stage}/{dataset}/{model_type}/'
    FILE_NAME = f'{model_type}_{exp_id}.py'

    if not os.path.exists(BASE_PATH):
        os.makedirs(BASE_PATH)

    code = m2c.export_to_python(classifier)
    with open(BASE_PATH + FILE_NAME, 'w') as f:
        f.writelines(code)
        
    return BASE_PATH + FILE_NAME

**Train Random Forest**

In [13]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# Directory to store the models and experiments
save_dir = './models/optimized/base/random_forest/'
os.makedirs(save_dir, exist_ok=True)

# Clear earlier saved files
for filename in os.listdir(save_dir):
    file_path = os.path.join(save_dir, filename)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f"Error while deleting file {file_path}: {e}")

# Prepare for saving results
results_file = './experiment_results.csv'
columns = ['model', 'features', 'feature_count', 'n_estimators', 'dataset_test_size', 'hz', 'data_set', 'quantization', 'accuracy', 'precision', 'recall', 'f1', 'model_path']

# Create or append to the results file
if not os.path.exists(results_file):
    df_results = pd.DataFrame(columns=columns)
else:
    df_results = pd.read_csv(results_file)

is_save_model = True
model_type = 'random_forest'
stage = 'optimized'
dataset = 'base'
quantization = None
estimators = None

cutoff = 0.99
dataset_test_sizes = [0.35]
datasets_setup = [(df_20hz, 20), (df_25hz, 25), (df_50hz, 50)]

experiment_results = []

for df_t in datasets_setup:
    for comb in comb_list:
        for dataset_test_size in dataset_test_sizes:
            df_filtered = df_t[0].filter(regex=comb)
            X_train, X_test, y_train, y_test = train_test_split(
                df_filtered.drop('label', axis=1), df_filtered['label'],
                test_size=dataset_test_size, random_state=42)

            for estimators in [4, 5, 6]:
                clf = RandomForestClassifier(n_jobs=-1, n_estimators=estimators, random_state=42)
                clf.fit(X_train, y_train)

                y_pred = clf.predict(X_test)

                accuracy = metrics.accuracy_score(y_test, y_pred)
                f1 = metrics.f1_score(y_test, y_pred, average='macro')
                precision = metrics.precision_score(y_test, y_pred, average='macro')
                recall = metrics.recall_score(y_test, y_pred, average='macro')

                if recall > cutoff:
                    signals = comb.replace('|label', '').split('|')
                    print(f"Signals: {signals} @ {df_t[1]} >> Acc: {accuracy}, Prec: {precision}, Recall: {recall}")

                    if is_save_model:
                        # Generate shorter model file name based on dataset and signals
                        signal_str = "_".join(signals)
                        model_filename = f"random_forest_{df_t[1]}_{signal_str}_{estimators}.py"
                        model_path = os.path.join(save_dir, model_filename)

                        # Create the Python code for the model
                        model_code = f"""
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import joblib

def create_model():
    clf = RandomForestClassifier(n_jobs=-1, n_estimators={estimators}, random_state=42)
    return clf

def evaluate_model(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred, average='macro')
    precision = metrics.precision_score(y_test, y_pred, average='macro')
    recall = metrics.recall_score(y_test, y_pred, average='macro')
    return accuracy, precision, recall, f1

# Example usage
if __name__ == '__main__':
    clf = create_model()
    accuracy, precision, recall, f1 = evaluate_model(clf, X_train, X_test, y_train, y_test)
    print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")
"""
                        # Save the Python code as a .py file
                        with open(model_path, 'w') as f:
                            f.write(model_code)

                        # Store the experiment result in a dictionary
                        experiment_result = {
                            'model': model_type,
                            'features': signals,
                            'feature_count': len(signals),
                            'n_estimators': estimators,
                            'dataset_test_size': dataset_test_size,
                            'hz': df_t[1],
                            'data_set': dataset,
                            'quantization': quantization,
                            'accuracy': accuracy,
                            'precision': precision,
                            'recall': recall,
                            'f1': f1,
                            'model_path': model_path
                        }

                        # Add the result to the list for later use
                        experiment_results.append(experiment_result)

                        # Save each experiment's result separately in the CSV
                        df_results = pd.concat([df_results, pd.DataFrame([experiment_result])], ignore_index=True)

# Save the results to CSV
df_results.to_csv(results_file, index=False)

# You can print or analyze the experiment results if needed
print("Filtered experiments with accuracy == 1.0:")
filtered_results = [exp for exp in experiment_results if exp['accuracy'] == 1.0]
for result in filtered_results:
    print(result)


Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y'] @ 20 >> Acc: 1.0, Prec: 1.0, Recall: 1.0
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y'] @ 20 >> Acc: 1.0, Prec: 1.0, Recall: 1.0
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y'] @ 20 >> Acc: 1.0, Prec: 1.0, Recall: 1.0
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_z'] @ 20 >> Acc: 1.0, Prec: 1.0, Recall: 1.0
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z'] @ 25 >> Acc: 0.9875, Prec: 0.984375, Recall: 0.9924242424242424
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z'] @ 25 >> Acc: 0.9875, Prec: 0.984375, Recall: 0.9924242424242424
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y'] @ 50 >> Acc: 1.0, Prec: 1.0, Recall: 1.0
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_z'] @ 50 >> Acc: 1.0, Prec: 1.0, Recall: 1.0
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z'] @ 50 >> Acc: 1.0, Prec: 1.0, Recall: 1.0
Filtered experiments with accuracy == 1.0:

**Train Desicion Tree**

In [10]:
import os
import csv
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import joblib  # Used for saving the model

# Custom CSV logger function
def log_experiment_csv(params, metrics, stage, dataset, model_type):
    log_dir = "logs"
    os.makedirs(log_dir, exist_ok=True)
    filename = f"{log_dir}/exp_log_{stage}_{dataset}_{model_type}.csv"

    log_data = {**params, **metrics}

    file_exists = os.path.isfile(filename)
    with open(filename, mode="a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=log_data.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(log_data)

# Save the model (function to save your model)
def save_model_optimized(model, stage, dataset, model_type, exp_id):
    model_dir = f"models/{stage}/{dataset}/{model_type}/"
    os.makedirs(model_dir, exist_ok=True)
    model_filename = f"{model_dir}/{exp_id}.pkl"
    joblib.dump(model, model_filename)
    return model_filename

# Main experiment logic
is_save_model = True
model_type = 'decision_tree'
stage = 'optimized'
dataset = 'base'
quantization = None
cutoff = 0.99
dataset_test_sizes = [0.35]
datasets_setup = [(df_20hz, 20), (df_25hz, 25), (df_50hz, 50)]

for df_t in datasets_setup:
    for comb in comb_list:
        for dataset_test_size in dataset_test_sizes:
            # Filter the dataset based on the signal combination
            df_filtered = df_t[0].filter(regex=comb)
            X_train, X_test, y_train, y_test = train_test_split(
                df_filtered.drop('label', axis=1), df_filtered['label'],
                test_size=dataset_test_size, random_state=42)

            # Create and train the Decision Tree model
            clf = DecisionTreeClassifier(random_state=42)
            clf.fit(X_train, y_train)

            # Predict and calculate metrics
            y_pred = clf.predict(X_test)

            accuracy = metrics.accuracy_score(y_test, y_pred)
            f1 = metrics.f1_score(y_test, y_pred, average='macro')
            precision = metrics.precision_score(y_test, y_pred, average='macro')
            recall = metrics.recall_score(y_test, y_pred, average='macro')

            # Only log and save the model if recall exceeds the cutoff
            if recall > cutoff:
                signals = comb.replace('|label', '').split('|')
                print(f"Signals: {signals} @ {df_t[1]} >> Acc: {accuracy}, Prec: {precision}, Recall: {recall}")

                if is_save_model:
                    exp_id = f"{model_type}_{df_t[1]}hz"
                    params = {
                        'exp_id': exp_id,
                        'model': model_type,
                        'features': signals,
                        'feature_count': len(signals),
                        'dataset_test_size': dataset_test_size,
                        'hz': df_t[1],
                        'data_set': dataset,
                        'quantization': quantization,
                        'other_params': 'default'
                    }

                    metrics_dict = {
                        'accuracy': accuracy,
                        'precision': precision,
                        'recall': recall,
                        'f1': f1
                    }

                    # Save the trained model
                    path = save_model_optimized(clf, stage, dataset, model_type, exp_id)
                    params['saved_model_path'] = path

                    # Log the experiment to the CSV file
                    log_experiment_csv(params, metrics_dict, stage, dataset, model_type)


Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x'] @ 25 >> Acc: 1.0, Prec: 1.0, Recall: 1.0
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_y'] @ 25 >> Acc: 1.0, Prec: 1.0, Recall: 1.0
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_z'] @ 25 >> Acc: 1.0, Prec: 1.0, Recall: 1.0


In [14]:
import os
import csv
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import joblib  # Used for saving the model

# Custom CSV logger function
def log_experiment_csv(params, metrics, stage, dataset, model_type, exp_id):
    log_dir = "logs"
    os.makedirs(log_dir, exist_ok=True)
    filename = f"{log_dir}/exp_log_{exp_id}.csv"  # Separate file per experiment using exp_id

    log_data = {**params, **metrics}

    # Writing experiment results to the separate file
    with open(filename, mode="w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=log_data.keys())
        writer.writeheader()
        writer.writerow(log_data)

# Save the model (function to save your model)
def save_model_optimized(model, stage, dataset, model_type, exp_id):
    model_dir = f"models/{stage}/{dataset}/{model_type}/"
    os.makedirs(model_dir, exist_ok=True)
    model_filename = f"{model_dir}/{exp_id}.pkl"
    joblib.dump(model, model_filename)
    return model_filename

# Main experiment logic
is_save_model = True
model_type = 'decision_tree'
stage = 'optimized'
dataset = 'base'
quantization = None
cutoff = 0.99
dataset_test_sizes = [0.35]
datasets_setup = [(df_20hz, 20), (df_25hz, 25), (df_50hz, 50)]

for df_t in datasets_setup:
    for comb in comb_list:
        for dataset_test_size in dataset_test_sizes:
            # Filter the dataset based on the signal combination
            df_filtered = df_t[0].filter(regex=comb)
            X_train, X_test, y_train, y_test = train_test_split(
                df_filtered.drop('label', axis=1), df_filtered['label'],
                test_size=dataset_test_size, random_state=42)

            # Create and train the Decision Tree model
            clf = DecisionTreeClassifier(random_state=42)
            clf.fit(X_train, y_train)

            # Predict and calculate metrics
            y_pred = clf.predict(X_test)

            accuracy = metrics.accuracy_score(y_test, y_pred)
            f1 = metrics.f1_score(y_test, y_pred, average='macro')
            precision = metrics.precision_score(y_test, y_pred, average='macro')
            recall = metrics.recall_score(y_test, y_pred, average='macro')

            # Only log and save the model if recall exceeds the cutoff
            if recall > cutoff:
                signals = comb.replace('|label', '').split('|')
                print(f"Signals: {signals} @ {df_t[1]} >> Acc: {accuracy}, Prec: {precision}, Recall: {recall}")

                if is_save_model:
                    exp_id = f"{model_type}_{df_t[1]}hz_{'_'.join(signals)}"
                    params = {
                        'exp_id': exp_id,
                        'model': model_type,
                        'features': signals,
                        'feature_count': len(signals),
                        'dataset_test_size': dataset_test_size,
                        'hz': df_t[1],
                        'data_set': dataset,
                        'quantization': quantization,
                        'other_params': 'default'
                    }

                    metrics_dict = {
                        'accuracy': accuracy,
                        'precision': precision,
                        'recall': recall,
                        'f1': f1
                    }

                    # Save the trained model
                    path = save_model_optimized(clf, stage, dataset, model_type, exp_id)
                    params['saved_model_path'] = path

                    # Log the experiment to the CSV file (separate file per experiment)
                    log_experiment_csv(params, metrics_dict, stage, dataset, model_type, exp_id)


Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x'] @ 25 >> Acc: 1.0, Prec: 1.0, Recall: 1.0
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_y'] @ 25 >> Acc: 1.0, Prec: 1.0, Recall: 1.0
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_z'] @ 25 >> Acc: 1.0, Prec: 1.0, Recall: 1.0


In [11]:
import os
import csv

# Function to filter experiments
def filter_experiments(log_dir, accuracy=1.0, model='random_forest'):
    matching_experiments = []
    for filename in os.listdir(log_dir):
        if filename.endswith('.csv'):
            file_path = os.path.join(log_dir, filename)
            with open(file_path, mode='r', newline='') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    if float(row['accuracy']) == accuracy and row['model'] == model:
                        matching_experiments.append(row)
    return matching_experiments

# Example usage
log_dir = "logs"  # Directory where logs are stored
filtered_experiments = filter_experiments(log_dir)

# Print the matching experiments
for exp in filtered_experiments:
    print(exp)


{'exp_id': 'random_forest_4_20hz', 'model': 'random_forest', 'features': "['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y']", 'feature_count': '5', 'n_estimators': '4', 'dataset_test_size': '0.35', 'hz': '20', 'data_set': 'base', 'quantization': '', 'other_params': 'default', 'saved_model_path': 'models/inf_time_test/random_forest/optimized/base\\random_forest_random_forest_4_20hz.py', 'accuracy': '1.0', 'precision': '1.0', 'recall': '1.0', 'f1': '1.0'}
{'exp_id': 'random_forest_5_20hz', 'model': 'random_forest', 'features': "['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y']", 'feature_count': '5', 'n_estimators': '5', 'dataset_test_size': '0.35', 'hz': '20', 'data_set': 'base', 'quantization': '', 'other_params': 'default', 'saved_model_path': 'models/inf_time_test/random_forest/optimized/base\\random_forest_random_forest_5_20hz.py', 'accuracy': '1.0', 'precision': '1.0', 'recall': '1.0', 'f1': '1.0'}
{'exp_id': 'random_forest_6_20hz', 'model': 'random_forest', 'features': "['acc_x', 'acc

In [12]:
import os
import csv

# Function to filter experiments based on accuracy and model type
def filter_experiments_by_model(log_dir, accuracy=1.0, model='decision_tree'):
    matching_experiments = []
    for filename in os.listdir(log_dir):
        if filename.endswith('.csv'):
            file_path = os.path.join(log_dir, filename)
            with open(file_path, mode='r', newline='') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    if float(row['accuracy']) == accuracy and row['model'] == model:
                        matching_experiments.append(row)
    return matching_experiments

# Example usage
log_dir = "logs"  # Directory where logs are stored
filtered_experiments = filter_experiments_by_model(log_dir)

# Print the matching experiments
for exp in filtered_experiments:
    print(exp)


{'exp_id': 'decision_tree_25hz', 'model': 'decision_tree', 'features': "['acc_x', 'acc_y', 'acc_z', 'gyro_x']", 'feature_count': '4', 'dataset_test_size': '0.35', 'hz': '25', 'data_set': 'base', 'quantization': '', 'other_params': 'default', 'saved_model_path': 'models/optimized/base/decision_tree/decision_tree_decision_tree_25hz.py', 'accuracy': '1.0', 'precision': '1.0', 'recall': '1.0', 'f1': '1.0'}
{'exp_id': 'decision_tree_25hz', 'model': 'decision_tree', 'features': "['acc_x', 'acc_y', 'acc_z', 'gyro_y']", 'feature_count': '4', 'dataset_test_size': '0.35', 'hz': '25', 'data_set': 'base', 'quantization': '', 'other_params': 'default', 'saved_model_path': 'models/optimized/base/decision_tree/decision_tree_decision_tree_25hz.py', 'accuracy': '1.0', 'precision': '1.0', 'recall': '1.0', 'f1': '1.0'}
{'exp_id': 'decision_tree_25hz', 'model': 'decision_tree', 'features': "['acc_x', 'acc_y', 'acc_z', 'gyro_z']", 'feature_count': '4', 'dataset_test_size': '0.35', 'hz': '25', 'data_set': '

In [7]:
import os
import csv
import m2cgen as m2c  # Changed from joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# Custom CSV logger function (unchanged)
def log_experiment_csv(params, metrics, stage, dataset, model_type):
    log_dir = "logs"
    os.makedirs(log_dir, exist_ok=True)
    filename = f"{log_dir}/exp_log_{stage}_{dataset}_{model_type}.csv"

    log_data = {**params, **metrics}

    file_exists = os.path.isfile(filename)
    with open(filename, mode="a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=log_data.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(log_data)

# Modified save function to use m2cgen
def save_model_optimized(model, stage, dataset, model_type, exp_id):
    model_dir = f"models/{stage}/{dataset}/{model_type}/"
    os.makedirs(model_dir, exist_ok=True)
    model_filename = f"{model_dir}/{exp_id}.py"  # Changed to .py
    
    # Convert model to Python code
    py_code = m2c.export_to_python(model)
    
    with open(model_filename, 'w') as f:
        f.write(py_code)
    
    return model_filename

# Main experiment logic (modified exp_id)
is_save_model = True
model_type = 'decision_tree'
stage = 'optimized'
dataset = 'base'
quantization = None
cutoff = 0.99
dataset_test_sizes = [0.35]
datasets_setup = [(df_20hz, 20), (df_25hz, 25), (df_50hz, 50)]

for df_t in datasets_setup:
    df, hz = df_t  # Unpack tuple for clarity
    for comb in comb_list:
        for dataset_test_size in dataset_test_sizes:
            # Filter the dataset based on the signal combination
            df_filtered = df.filter(regex=comb)
            X_train, X_test, y_train, y_test = train_test_split(
                df_filtered.drop('label', axis=1), df_filtered['label'],
                test_size=dataset_test_size, random_state=42)

            # Create and train the Decision Tree model
            clf = DecisionTreeClassifier(random_state=42)
            clf.fit(X_train, y_train)

            # Predict and calculate metrics
            y_pred = clf.predict(X_test)
            accuracy = metrics.accuracy_score(y_test, y_pred)
            f1 = metrics.f1_score(y_test, y_pred, average='macro')
            precision = metrics.precision_score(y_test, y_pred, average='macro')
            recall = metrics.recall_score(y_test, y_pred, average='macro')

            if recall > cutoff:
                signals = comb.replace('|label', '').split('|')
                print(f"Signals: {signals} @ {hz}Hz >> Acc: {accuracy:.2f}, Prec: {precision:.2f}, Recall: {recall:.2f}")

                if is_save_model:
                    # Generate unique experiment ID with frequency and signals
                    exp_id = f"{model_type}_{hz}hz_{'_'.join(signals)}"
                    
                    params = {
                        'exp_id': exp_id,
                        'model': model_type,
                        'features': signals,
                        'feature_count': len(signals),
                        'dataset_test_size': dataset_test_size,
                        'hz': hz,
                        'data_set': dataset,
                        'quantization': quantization,
                        'other_params': 'default'
                    }

                    metrics_dict = {
                        'accuracy': accuracy,
                        'precision': precision,
                        'recall': recall,
                        'f1': f1
                    }

                    # Save as Python file
                    path = save_model_optimized(clf, stage, dataset, model_type, exp_id)
                    params['saved_model_path'] = path

                    # Log the experiment
                    log_experiment_csv(params, metrics_dict, stage, dataset, model_type)


Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_x'] @ 25Hz >> Acc: 1.00, Prec: 1.00, Recall: 1.00
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_y'] @ 25Hz >> Acc: 1.00, Prec: 1.00, Recall: 1.00
Signals: ['acc_x', 'acc_y', 'acc_z', 'gyro_z'] @ 25Hz >> Acc: 1.00, Prec: 1.00, Recall: 1.00
